# Final Project: Web Service APIs
## Data acquisition notebook
by Lin Ma and Khanh Tran

This note book continue with the user_token.json file gathered from the first notebook. It starts from getting the premise of a valid token. So if the access token we gathered originally from the user is expired or not, we refresh the token before the further steps. 
What needs to be done in this notebook include: 
- Step 1: Refresh the expired token
- Step 2: Data acquirization: Intacting with the Spotify web API to acquiring intresting data
- Step 3: Data normalization: Organzing the data into the normalized dataframe 
- Step 4: Relational Database: Constructing a sound relational database based on the values stored in dataframe.

Other than importing, the following code is doing the Step 1, which is just mapping the users with new tokens.

## Challenges & Success

One of the challeges that we faced is setting the right data type for columns in our table. Because our users include international students, we have many artist names and album names that are not in English. After running into the problem of not being able to push some artist/ album name to our database's tables, we figured that we should change the data type to NVARCHAR, which can handle other languages apart from English.

Another challenge that we managed to overcome is decompositing functions to handle pages of an http response in a clean manner. At first, we had duplicates in our acquired data due to the wrong offset (0, 50, 100, 150, etc). Our default limit for all the requests with a limit parameter is 50. Therefore, if there are more than one page, the offset should be 0, 51, 101, 151 and such.

Last but not least, one of a significant problem that we came accross was generating the random id for our users, artists and albums. Because the random module we imported was not completely random, we ended up having to create an idList to keep track of the numbers already generated, and wrote a recursive function to regenerate a random number if the previous one generated is already in the list.

In [75]:
import requests
import json
import pandas as pd
import random, string
import sqlalchemy as sa
from random import randint

## Step 1: Refresh the expired token

In [76]:
def extractJSON(filename):
    """ This function takes a json file to read from then returns the dictionary in that file.
    Parameter:
        filename: a local file's name
    Return: a dictionary
    """
    with open(filename) as file:
        allcreds = json.load(file)
        return allcreds

In [77]:
def db_setup(user, password, database):
    """ This function takes in SQL user & password credentials to establish a connection to the server.
    Parameter:
        user: an user
        password: a password
        database: the database to create connection with
    Return:
        e: an engine created to establish connection
        c: a connection to the server
        cstring: the protocol to connect to the SQL server
    """
    template = 'mysql+mysqlconnector://{}:{}@hadoop2.mathsci.denison.edu/{}'
    cstring = template.format(user, password, database)
    e = sa.create_engine(cstring)
    c = e.connect()
    return e, c, cstring

In [17]:
def getNewToken(tokenmap, accessurl):
    """ This function takes in a dictionary of token and makes HTTP request to refresh & update the tokens in that dictionary
    Parameters:
        tokenmap: a dictionary containing users' tokens
        accessurl: a url to access Spotify API
    Return: the updated dictionary of users' tokens
    """
    for user in tokenmap:
        h = {"Authorization": "Basic" + " "+ tokenmap[user]['access_token']}
        d ={}
        d['grant_type']= 'refresh_token'
        d['refresh_token'] = tokenmap[user]['refresh_token']
        newresp = requests.post(accessurl, data=d, headers=h)    
        retval1 = newresp.json()
        if 'access_token' in retval1:
            tokenmap[user]['access_token'] =retval1['access_token']
        if 'token_type' in retval1:
            tokenmap[user]['token_type'] =retval1['token_type']
        if 'expires_in' in retval1:
            tokenmap[user]['expires_in'] =retval1['expires_in']
        if 'scope' in retval1:
            tokenmap[user]['scope'] =retval1['scope']
    return tokenmap

## Step 2,3,4: Data acquirization & normalization + Relational Database

### Create Tables

In [18]:
def createTables(table, connection):
    """ This function takes in a connection and a table name that can be create in the SQL query
        Parameter table a table need in the SQL query 
        connection the SQL connection created in the previous founction
        Returns: None 
    """
    createUsers = """CREATE TABLE Users(
    UserID VARCHAR (20)     NOT NULL,
    UserName VARCHAR (10)     NOT NULL,
    NumTrack INT              NOT NULL,
    PRIMARY KEY (UserID)
    );"""
    
    createUser_Artist = """CREATE TABLE User_Artist(
    UserID VARCHAR (20)     NOT NULL,
    ArtistID VARCHAR (20)     NOT NULL,
    PRIMARY KEY (UserID, ArtistID)
    );"""
    
    createArtists = """CREATE TABLE Artists(
    ArtistID VARCHAR (20)     NOT NULL,
    ArtistName VARCHAR (40)   NOT NULL,
    FollowersInMillions DECIMAL(4,2),
    Popularity INT,
    PRIMARY KEY (ArtistID)
    );"""
    
    createArtist_Album = """CREATE TABLE Artist_Album(
    ArtistID VARCHAR (20)     NOT NULL,
    AlbumID VARCHAR (20)      NOT NULL,
    PRIMARY KEY (ArtistID, AlbumID)
    );"""
    
    createAlbums = """CREATE TABLE Albums(
    AlbumID VARCHAR (20)      NOT NULL,
    AlbumName NVARCHAR (50)    NOT NULL,
    ReleaseDate DATE,
    PRIMARY KEY (AlbumID)
    );"""
    
    createUANames = """CREATE TABLE UANames(
    UserName VARCHAR (10)    NOT NULL,
    ArtistName VARCHAR (40)  NOT NULL,
    PRIMARY KEY (UserName, ArtistName)
    );"""
    
    createAANames = """CREATE TABLE AANames(
    ArtistName VARCHAR (40)    NOT NULL,
    AlbumName NVARCHAR (50)  NOT NULL,
    PRIMARY KEY (ArtistName, AlbumName)
    );"""
    
    createTable = 'create'+table
    connection.execute(createTable)

### Insert into tables
#### Users Table

In [19]:
def randomID(idList):
    """ This function takes in list of id and randomlize it with to create a random list of id which can never repeat
    Parameters:
        idList: the list of id number generated
    Return: the random string of number generated
    """
    i = str(random.randrange(1, 200000000, 1))
    if i not in idList:
        return i
    return randomID(idList)

In [20]:
def urlContruct(scope):
    """ This function construct the url which take in the scopes
    Parameters:
        scope that has been defined in the json file
    Return: url a valid url that provides infomations with desired scope
    """
    protocol = "https://"
    location = "api.spotify.com/v1/me"
    resource = "/" +scope
    template = '{}{}{}'
    url = template.format(protocol, location, resource)
    return url

In [21]:
def getTrack(tokenmap, user, offset):
    """ This function get the tracks from the url takes in tokenmap user and offset
    Parameters:
    tokenmap user with maped token as a dictionary
    user individual user in tokenmap
    offset starting item of each page

    Return: resp.json(): json file like response from the url
    """
    url = urlContruct("tracks")
    d = {}
    d["limit"] = 50
    d["offset"] = offset
    h = {"Authorization": "Bearer" + " "+ tokenmap[user]['access_token']}
    resp = requests.get(url, params = d, headers= h)
    return resp.json()

In [22]:
def UsersTable(tokenmap, connection):
    """This function takes in a connection and a table name that can be create in the SQL query form a users table
        Parameter table a table need in the SQL query 
        connection the SQL connection created in the previous founction
        Returns: NONE
    """
    insertUsers = sa.sql.text("""
    INSERT INTO Users (UserID, UserName, NumTrack)  
    VALUES (:ui, :u, :nt);
    """)
    idList = []
    for user in tokenmap:
        i = randomID(idList)
        idList.append(i)
        boundInsertUsers = insertUsers.bindparams(ui= 'US'+i, u= user, nt= getTrack(tokenmap, user, 0)['total'])
        resultproxy = connection.execute(boundInsertUsers)

#### Artists Table

In [27]:
def toDataFrame(DICT,indexCol):
    """ This function takes in a dictionary and use pandas
    to yield a dataframe from that dictionary.
    Parameters:
        DICT: a dictionary
    Return: a dataframe
    """
    df = pd.DataFrame(DICT) #use pandas to yield a dataframe from DICT
    df.drop_duplicates(inplace = True)
    df.set_index(indexCol, inplace = True)
    return df #return the dataframe

In [28]:
def concatDataFrame(df1, df2):
    """ This function takes in dataframe 1 and dataframe 2 concat the data frame 
    Parameters:
        df1 a dataframe
        df2 another dataframe
    Return: a dataframe
    """
    result = pd.concat([df1, df2], axis=1, join='outer')
    result = result.reset_index()
    result = result.astype(object).where(pd.notnull(result), None)
    return result

In [29]:
def getArtist(tokenmap, user):
    url = urlContruct("following")
    d = {}
    d["limit"] = 50
    d["type"] = "artist"
    h = {"Authorization": "Bearer" + " "+ tokenmap[user]['access_token']}
    resp = requests.get(url, params = d, headers= h)
    return resp.json()['artists']['items']

In [30]:
def extractArtist(tokenmap):
    """This function takes in a connection and a table name that can be create in the SQL query form a users table
        Parameter table a table need in the SQL query 
        connection the SQL connection created in the previous founction
        Returns: NONE
    """
    u = []
    a = []
    p = []
    for user in tokenmap:
        for artist in getArtist(tokenmap, user):
            u.append(user)
            a.append(artist['name'])
            p.append(artist['popularity'])
    return u, a, p

In [31]:
def createDictArtists(tokenmap):
    """This function takes in a connection and a table name that can be create in the SQL query form a users table
        Parameter table a table need in the SQL query 
        connection the SQL connection created in the previous founction
        Returns: NONE
    """
    u, a, p = extractArtist(tokenmap)
    ap = {}
    ap['Artist'] = a
    ap['Popularity'] = p
    
    ua = {}
    ua['UserName'] = u
    ua['ArtistName'] = a
    
    adict = [ap,ua]
    return adict

In [32]:
def ArtistsDataFrame(tokenmap):
    """This function takes in a connection and a table name that can be create in the SQL query form a users table
        Parameter table a table need in the SQL query 
        connection the SQL connection created in the previous founction
        Returns: NONE
    """  
    df1 = toDataFrame(extractJSON("MergeArtist.json"),'Artist').drop(['Rank','Country'], axis=1, inplace = True)

    ap = createDictArtists(tokenmap)[0]
    df2 = toDataFrame(ap, 'Artist')
    result = concatDataFrame(df1, df2)
    return result

In [33]:
def ArtistsTable(tokenmap, connection):
    """This function takes in a connection and a table name that can be create in the SQL query form a users table
        Parameter table a table need in the SQL query 
        connection the SQL connection created in the previous founction
        Returns: NONE
    """
    df = ArtistsDataFrame(tokenmap)
    insertArtists = sa.sql.text("""
    INSERT INTO Artists (ArtistID, ArtistName, FollowersInMillions,Popularity)  
    VALUES (:aid, :an, :f, :p);
    """)
    idList = []
    for index, row in df.iterrows():
        i = randomID(idList)
        idList.append(i)
        boundInsertArtists = insertArtists.bindparams(aid='ART'+i, an=row['index'], f=row['Followers (millions)'], p=row['Popularity'])
        resultproxy = connection.execute(boundInsertArtists)

#### User_Artist Table

In [35]:
def UANamesDataFrame(tokenmap):
    """This function takes in a connection and a table name that can be create in the SQL query form a users table
        Parameter table a table need in the SQL query 
        connection the SQL connection created in the previous founction
        Returns: NONE
    """
    ua = createDictArtists(tokenmap)[1]
    df3 = toDataFrame(ua,"None")

In [36]:
def UANamesTable(tokenmap, connection):
    """This function takes in a connection and a table name that can be create in the SQL query form a users table
        Parameter table a table need in the SQL query 
        connection the SQL connection created in the previous founction
        Returns: NONE
    """
    df = UANamesDataFrame(tokenmap)
    insertUANames = sa.sql.text("""
    INSERT INTO UANames (UserName, ArtistName)  
    VALUES (:un, :an);
    """)
    for index, row in df.iterrows():
        boundInsertUANames = insertUANames.bindparams(un = row['UserName'], an=row['ArtistName'])
        resultproxy = connection.execute(boundInsertUANames)

In [37]:
def UserArtistDataFrame(connection):
    """This function takes in a connection and a table name that can be create in the SQL query form a users table
        Parameter table a table need in the SQL query 
        connection the SQL connection created in the previous founction
        Returns: NONE
    """
    query = """
    SELECT Users.UserID, Artists.ArtistID
    FROM UANames INNER JOIN Users
        ON UANames.UserName = Users.UserName
    INNER JOIN Artists
        ON UANames.ArtistName = Artists.ArtistName
    ORDER BY Users.UserName, Artists.ArtistName
    """
    result = connection.execute(query)
    resultdf = result.DataFrame()
    return resultdf

In [38]:
def UserArtistTable(tokenmap, connection):
    """This function takes in a connection and a table name that can be create in the SQL query form a users table
        Parameter table a table need in the SQL query 
        connection the SQL connection created in the previous founction
        Returns: NONE
    """
    df = UserArtistDataFrame(connection)
    insertUserArtist = sa.sql.text("""
    INSERT INTO User_Artist (UserID, ArtistID)  
    VALUES (:uid, :aid);
    """)
    for index, row in df.iterrows():
        boundInsertUserArtist = insertUserArtist.bindparams(uid= row['UserID'],aid=row['ArtistID'])
        resultproxy = connection.execute(boundInsertUserArtist)

#### Albums Table

In [41]:
def getAlbum(resp):
    """This function takes in a connection and a table name that can be create in the SQL query form a users table
        Parameter table a table need in the SQL query 
        connection the SQL connection created in the previous founction
        Returns: NONE
    """
    alName = []
    reDate = []
    arName = []
    for i in resp['items']:
        al = i['track']['album']
        ar = i['track']['artists']
        if len(ar) == 1:
            alName.append(al['name'])
            reDate.append(al['release_date'])
            arName.append(ar[0]['name'])
        else:
            for a in range(len(ar)):
                alName.append(al['name'])
                reDate.append(al['release_date'])
                arName.append(ar[a]['name'])
    return alName, reDate, arName

In [42]:
def processAlbum(tokenmap):
    """This function takes in a connection and a table name that can be create in the SQL query form a users table
        Parameter table a table need in the SQL query 
        connection the SQL connection created in the previous founction
        Returns: NONE
    """
    alName = []
    reDate = []
    arName = []
    for user in tokenmap:
        resp = getTrack(tokenmap, user, 50, 0)
        pages = resp['total']//50
        al, re, ar = getAlbum(resp)
        alName = alName + al
        reDate = reDate + re
        arName = arName + ar
        for p in range(pages):
            offset = 50*(p+1) + 1
            resp = getTrack(tokenmap, user, 50, offset)
            al, re, ar = getAlbum(resp)
            alName = alName + al
            reDate = reDate + re
            arName = arName + ar
    return alName, reDate, arName

In [43]:
def createDictAlbums(tokenmap):
    """This function takes in a connection and a table name that can be create in the SQL query form a users table
        Parameter table a table need in the SQL query 
        connection the SQL connection created in the previous founction
        Returns: NONE
    """
    alName, reDate, arName = processAlbum(tokenmap)
    d1 = {}
    d1['AlbumName'] = alName
    d1['ReleaseDate'] = reDate
    d2 = {}
    d2['AlbumName'] = alName
    d2['ArtistName'] = arName
    return d1,d2

In [44]:
def AlbumsDataFrame(tokenmap):
    """This function takes in a connection and a table name that can be create in the SQL query form a users table
        Parameter table a table need in the SQL query 
        connection the SQL connection created in the previous founction
        Returns: NONE
    """
    d1, d2 = createDictAlbums(tokenmap)
    df1 = toDataFrame(d1, "None")
    df2 = toDataFrame(d2, "None")
    df = [df1, df2]
    return df

In [45]:
def AlbumsTable(tokenmap, connection):
    """This function takes in a connection and a table name that can be create in the SQL query form a users table
        Parameter table a table need in the SQL query 
        connection the SQL connection created in the previous founction
        Returns: NONE
    """
    df = AlbumsDataFrame(tokenmap)[0]
    insertAlbums = sa.sql.text("""
    INSERT INTO Albums (AlbumID, AlbumName, ReleaseDate)  
    VALUES (:aid, :an, :rd);
    """)
    idList = []
    for index, row in df.iterrows():
        i = randomID(idList)
        idList.append(i)
        boundInsertAlbums = insertAlbums.bindparams(aid= 'ALB'+i, an=row['AlbumName'] ,rd= row['ReleaseDate'])
        resultproxy = connection.execute(boundInsertAlbums)

#### Artist_Album Table

In [47]:
def AANamesTable(tokenmap, connection):
    """This function takes in a connection and a table name that can be create in the SQL query form a users table
        Parameter table a table need in the SQL query 
        connection the SQL connection created in the previous founction
        Returns: NONE
    """
    df = AlbumsDataFrame(tokenmap)[1]
    insertAANames = sa.sql.text("""
    INSERT INTO AANames (ArtistName, AlbumName)  
    VALUES (:ar, :al);
    """)
    for index, row in df.iterrows():
        boundInsertAANames = insertAANames.bindparams(ar = row['ArtistName'], al=row['AlbumName'])
        resultproxy = connection.execute(boundInsertAANames)

In [48]:
def ArtistAlbumDataFrame(connection):
    """This function takes in a connection and a table name that can be create in the SQL query form a users table
        Parameter table a table need in the SQL query 
        connection the SQL connection created in the previous founction
        Returns: NONE
    """
    query = """
    SELECT Artists.ArtistID, Albums.AlbumID
    FROM AANames INNER JOIN Artists
        ON AANames.ArtistName = Artists.ArtistName
    INNER JOIN Albums
        ON AANames.AlbumName = Albums.AlbumName
    ORDER BY Artists.ArtistName, Albums.AlbumName
    """
    result = connection.execute(query)
    resultdf = result.DataFrame()
    return resultdf

In [49]:
def ArtistAlbumTable(tokenmap, connection):
    """This function takes in a connection and a table name that can be create in the SQL query form a users table
        Parameter table a table need in the SQL query 
        connection the SQL connection created in the previous founction
        Returns: NONE
    """
    df = ArtistAlbumDataFrame(connection)
    insertArtistAlbum = sa.sql.text("""
    INSERT INTO Artist_Album (ArtistID, AlbumID)  
    VALUES (:ari, :ali);
    """)
    for index, row in df.iterrows():
        boundInsertArtistAlbum = insertArtistAlbum.bindparams(ari= row['ArtistID'],ali=row['AlbumID'])
        resultproxy = connection.execute(boundInsertArtistAlbum)

#### The next functions are examples of the SQL query we used to pull from the database to Tableau in order to create visualizations.

#### Question 1:  Is there a linear relationship found between CS 181 Spotify users(n=18) saved tracks and the number of following artists?

In [78]:
def question1(connection):
    """ This function create an SQL query to get the data for question 1 by joining Artists and User_Artist Tables
    Parameter:
        connection a SQL connection
    Return: q1
    """
    query ="""
    SELECT Artists.ArtistName, COUNT(User_Artist.UserID) as Followers
    FROM User_Artist INNER JOIN Artists
        ON User_Artist.ArtistID = Artists.ArtistID
    GROUP BY User_Artist.ArtistID
    ORDER BY Followers DESC
    LIMIT 20
    """
    t1 = connection.execute(query)
    q1 = t1.fetchall()
    return q1

#### Question 2: How does CS181’s TOP 20 artists differ from Spotify’s general TOP 20?

In [79]:
def question2(connection):
    """ This function create an SQL query to get the data for question 2 by joining Users and User_Artist Tables
    Parameter:
        connection: a SQL connection
    Return: q2
    """
    k="""
    SELECT Users.UserID, Users.numTrack, COUNT(User_Artist.ArtistID) as numArtist
    FROM User_Artist INNER JOIN Users
        ON User_Artist.UserID = Users.UserID
    GROUP BY User_Artist.UserID
    """
    t2 = connection.execute(query)
    q2 = t2.fetchall()
    return q2

#### Question 3: For the Artists that CS 181 Spotify users are following: Is there a trend observed in the artist’s popularity and the release date of their newest album that we listen to?

In [70]:
def question3(connection):
    """ This function create an SQL query to get the data for question 3 by joining 3 tables
    Artists, User_Artist and Users Tables.
    Parameter:
        connection: a SQL connection
    Return: q3
    """
    query="""
    SELECT Artists.ArtistName, Artists.Popularity, MAX(Albums.ReleaseDate)
    FROM Artist_Album INNER JOIN Artists
        ON Artist_Album.ArtistID = Artists.ArtistID
    INNER JOIN Albums
        ON Artist_Album.AlbumID = Albums.AlbumID
    WHERE (Artists.Popularity IS NOT NULL)
    GROUP BY Artists.ArtistID
    ORDER BY Artists.Popularity DESC
    """
    t3 = connection.execute(query)
    q3 = t3.fetchall()
    return q3

#### The main function

In [80]:
def main():
    """ This main function takes all the functions that defined abrove and print the tables
    """
    sqlcreds = extractJSON("creds_SQL.json")['mysql']
    engine, connection, cstring = db_setup(sqlcreds['user'], sqlcreds['password'], sqlcreds['user'])
    
    #refresh token
    creds = extractJSON("creds.json")['spotify']
    accessurl = "https://accounts.spotify.com/api/token"
    tokenmap= extractJSON("user_token.json")
    getNewToken(tokenmap, accessurl, creds)    
    
    #Users Table
    #createTables("Users", connection)
    #UsersTable(tokenmap, connection)
    
    #Artists Table
    #createTables("Artists", connection)
    #ArtistsTable(tokenmap, connection)
    
    #User_Artist Table
    #createTables("User_Artist", connection)
    #UserArtistTable(tokenmap, connection)
    
    #Albums Table
    #createTables("Albums", connection)
    #AlbumsTable(tokenmap, connection)
    
    #Artist_Album Table
    #createTables("Artist_Album", connection)
    #ArtistAlbumTable(tokenmap, connection)

    connection.close()
    del engine

In [81]:
main()